# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core import Workspace, Experiment, Environment, Model
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

from azureml.core import Webservice
# Check core SDK version number


import requests
import json
print("SDK version:", azureml.core.VERSION)

SDK version: 1.28.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'EXP_HEART_FAILURE_AML'

experiment=Experiment(ws, experiment_name)

In [3]:
# Building CPU cluster
amlcompute_cluster_name = "common-gy"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=4,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
found = False
key = "heart-failure" #"Heart failure"
description_text = "Heart failure dataset for udacity capstone"

if key in ws.datasets.keys():
        found = True
        print("Found dataset: ",key)
        dataset = ws.datasets[key]

if not found:
        # Create AML Dataset and register it into Workspace
        print("Registering dataset: ",key)
        example_data = "./data/heart_failure_clinical_records_dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(example_data)
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

Found dataset:  heart-failure


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,0.43,581.84,0.42,38.08,0.35,263358.03,1.39,136.63,0.65,0.32,130.26,0.32
std,11.89,0.50,970.29,0.49,11.83,0.48,97804.24,1.03,4.41,0.48,0.47,77.61,0.47
min,40.00,0.00,23.00,0.00,14.00,0.00,25100.00,0.50,113.00,0.00,0.00,4.00,0.00
25%,51.00,0.00,116.50,0.00,30.00,0.00,212500.00,0.90,134.00,0.00,0.00,73.00,0.00
50%,60.00,0.00,250.00,0.00,38.00,0.00,262000.00,1.10,137.00,1.00,0.00,115.00,0.00
75%,70.00,1.00,582.00,1.00,45.00,1.00,303500.00,1.40,140.00,1.00,1.00,203.00,1.00
max,95.00,1.00,7861.00,1.00,80.00,1.00,850000.00,9.40,148.00,1.00,1.00,285.00,1.00


In [8]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,0,582,0,20,1,265000.00,1.90,130,1,0,4,1
1,55.00,0,7861,0,38,0,263358.03,1.10,136,1,0,6,1
2,65.00,0,146,0,20,0,162000.00,1.30,129,1,1,7,1
3,50.00,1,111,0,20,0,210000.00,1.90,137,1,0,7,1
4,65.00,1,160,1,20,0,327000.00,2.70,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
project_folder='./realtime_automl'
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             model_explainability = True,
                             enable_onnx_compatible_models=True,
                             **automl_settings
                            )

In [10]:
automl_run1 = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on common-gy with default configuration
Running on remote compute: common-gy


Experiment,Id,Type,Status,Details Page,Docs Page
EXP_HEART_FAILURE_AML,AutoML_e0801f1a-6e7e-448a-af65-866745c60af5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

In [10]:
rns = experiment.get_runs()
next(rns)

Experiment,Id,Type,Status,Details Page,Docs Page
EXP_HEART_FAILURE_AML,AutoML_03202041-e1a9-4c4a-b446-9912b5836698,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
widget1 = RunDetails(automl_run1)
widget1.show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_run = automl_run1.get_best_child()
best_run.get_metrics()

{'f1_score_macro': 0.8183047177463653,
 'f1_score_micro': 0.8493103448275864,
 'f1_score_weighted': 0.8466029332405407,
 'recall_score_weighted': 0.849310344827586,
 'average_precision_score_micro': 0.9280137323683842,
 'balanced_accuracy': 0.8201338487027845,
 'AUC_macro': 0.9170805630677485,
 'log_loss': 0.5109356686600777,
 'AUC_weighted': 0.9170805630677485,
 'norm_macro_recall': 0.6402676974055691,
 'accuracy': 0.849310344827586,
 'precision_score_micro': 0.849310344827586,
 'recall_score_micro': 0.849310344827586,
 'weighted_accuracy': 0.867837401542268,
 'average_precision_score_macro': 0.906866706898309,
 'average_precision_score_weighted': 0.930914983621917,
 'recall_score_macro': 0.8201338487027845,
 'precision_score_macro': 0.8351928380656004,
 'matthews_correlation': 0.6539797696541126,
 'precision_score_weighted': 0.8610438424562246,
 'AUC_micro': 0.9235694279297133,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_e0801f1a-6e7e-448a-af65-866745c60af5_36/accu

In [13]:
print('Besr run id : ', best_run.id)

Besr run id :  AutoML_e0801f1a-6e7e-448a-af65-866745c60af5_36


In [14]:
#TODO: Save the best model
best_automl_model, onnx_model = automl_run1.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, './automl_model.onnx')

In [15]:
best_run, fitted_model = automl_run1.get_output()
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('randomforestclassifier', RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None, criterion='entropy', max_depth=None, max_features='log2', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01, min_samples_split=0.01, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False))], verbose=False)), ('30', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, clas

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
model_onnx = best_run.register_model(model_path='outputs/model.onnx', model_name='mdl-heart-failure-automl-onnx-1')

In [17]:
model_pkl = best_run.register_model(model_path='outputs/model.pkl', model_name='mdl-heart-failure-automl-pkl-1')

In [18]:
run_env = best_run.get_environment()
print(run_env)

Environment(Name: AutoML-AzureML-AutoML,
Version: Autosave_2021-05-30T12:31:38Z_18564f47)


In [19]:

#AzureML-Scikit-learn0.24-Cuda11-OpenMpi4.1.0-py36
#myenv = Environment.get(workspace=ws, name="AzureML-Tutorial")
script_path = './scoring_file_v_2_0_0.py'
inference_config = InferenceConfig(entry_script=script_path, 
environment=run_env)

In [20]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

In [21]:
service = Model.deploy(
    workspace=ws,
    name="mdldply-heart-failure-pkl-aci",
    models=[model_pkl],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-30 14:04:22+00:00 Creating Container Registry if not exists.
2021-05-30 14:04:22+00:00 Registering the environment.
2021-05-30 14:04:22+00:00 Use the existing image.
2021-05-30 14:04:22+00:00 Generating deployment configuration.
2021-05-30 14:04:23+00:00 Submitting deployment to compute.
2021-05-30 14:04:26+00:00 Checking the status of deployment mdldply-heart-failure-pkl-aci..
2021-05-30 14:08:50+00:00 Checking the status of inference endpoint mdldply-heart-failure-pkl-aci.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [19]:
#name="mdldply-heart-failure-pkl-aci"
#service = Webservice(name=name, workspace=ws)
#service.update(enable_app_insights=True)

In [27]:
script_path_onnx = './scoring_file_v_2_0_0_onnx.py'
inference_config_onnx = InferenceConfig(entry_script=script_path_onnx, 
environment=run_env)

service_onnx = Model.deploy(
    workspace=ws,
    name="mdldply-heart-failure-onnx-aci",
    models=[model_pkl],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)
service_onnx.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-30 14:33:42+00:00 Creating Container Registry if not exists.
2021-05-30 14:33:42+00:00 Registering the environment.
2021-05-30 14:33:43+00:00 Use the existing image.
2021-05-30 14:33:43+00:00 Generating deployment configuration.
2021-05-30 14:33:44+00:00 Submitting deployment to compute.
2021-05-30 14:33:47+00:00 Checking the status of deployment mdldply-heart-failure-onnx-aci..
2021-05-30 14:37:53+00:00 Checking the status of inference endpoint mdldply-heart-failure-onnx-aci.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
data_json = {
	"data":
	[
		{"age": 75, "anaemia":0, "creatinine_phosphokinase": 582, 
		"diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 1, 
		"platelets": 265000, "serum_creatinine": 1.9, "serum_sodium": 130, "sex": 1, "smoking": 0, "time":4}
	]

}

In [30]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True)

data = {
    "Inputs": {
        "data":
        [
            {
                'age': "75",
                'anaemia': "0",
                'creatinine_phosphokinase': "0",
                'diabetes': "582",
                'ejection_fraction': "80",
                'high_blood_pressure': "1",
                'platelets': "265000",
                'serum_creatinine': "1.9",
                'serum_sodium': "130",
                'sex': "0",
                'smoking': "0",
                'time': "265",
            },
        ],
    }
}

body = str.encode(json.dumps(data))

url = 'http://d56d611c-6714-480a-a0aa-91bf6a1cb78d.southcentralus.azurecontainer.io/score'
api_key = 'O7uaZBQfJw29Jgy4DEtCsWzVhArgQcwO' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'[0]'


TODO: In the cell below, print the logs of the web service and delete the service

In [29]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2021-05-30T14:08:37,666417400+00:00 - iot-server/run 
2021-05-30T14:08:37,676832100+00:00 - gunicorn/run 
2021-05-30T14:08:37,690341100+00:00 - rsyslog/run 
2021-05-30T14:08:37,702829700+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [ ]:
compute_target.delete()